In [13]:
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

import collections
import os

import time


import numpy as np
import tensorflow as tf
from unicodedata import normalize


In [2]:
# """Utilities for parsing PTB text files."""

# def _read_words(filename):
#   with tf.gfile.GFile(filename, "r") as f:
#     return f.read().replace("\n", "<eos>").split()


# def _build_vocab(filename):
#   data = _read_words(filename)

#   counter = collections.Counter(data)
#   count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

#   words, _ = list(zip(*count_pairs))
#   word_to_id = dict(zip(words, range(len(words))))

#   return word_to_id


# def _file_to_word_ids(filename, word_to_id):
#   data = _read_words(filename)
#   return [word_to_id[word] for word in data]


# def ptb_raw_data(data_path=None):
#   """Load PTB raw data from data directory "data_path".

#   Reads PTB text files, converts strings to integer ids,
#   and performs mini-batching of the inputs.

#   The PTB dataset comes from Tomas Mikolov's webpage:

#   http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz

#   Args:
#     data_path: string path to the directory where simple-examples.tgz has
#       been extracted.

#   Returns:
#     tuple (train_data, valid_data, test_data, vocabulary)
#     where each of the data objects can be passed to PTBIterator.
#   """

#   train_path = os.path.join(data_path, "ptb.train.txt")
#   valid_path = os.path.join(data_path, "ptb.valid.txt")
#   test_path = os.path.join(data_path, "ptb.test.txt")

#   word_to_id = _build_vocab(train_path)
#   train_data = _file_to_word_ids(train_path, word_to_id)
#   valid_data = _file_to_word_ids(valid_path, word_to_id)
#   test_data = _file_to_word_ids(test_path, word_to_id)
#   vocabulary = len(word_to_id)
#   return train_data, valid_data, test_data, vocabulary


# def ptb_iterator(raw_data, batch_size, num_steps):
#   """Iterate on the raw PTB data.

#   This generates batch_size pointers into the raw PTB data, and allows
#   minibatch iteration along these pointers.

#   Args:
#     raw_data: one of the raw data outputs from ptb_raw_data.
#     batch_size: int, the batch size.
#     num_steps: int, the number of unrolls.

#   Yields:
#     Pairs of the batched data, each a matrix of shape [batch_size, num_steps].
#     The second element of the tuple is the same data time-shifted to the
#     right by one.

#   Raises:
#     ValueError: if batch_size or num_steps are too high.
#   """
#   raw_data = np.array(raw_data, dtype=np.int32)

#   data_len = len(raw_data)
#   batch_len = data_len // batch_size
#   data = np.zeros([batch_size, batch_len], dtype=np.int32)
#   for i in range(batch_size):
#     data[i] = raw_data[batch_len * i:batch_len * (i + 1)]

#   epoch_size = (batch_len - 1) // num_steps

#   if epoch_size == 0:
#     raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

#   for i in range(epoch_size):
#     x = data[:, i*num_steps:(i+1)*num_steps]
#     y = data[:, i*num_steps+1:(i+1)*num_steps+1]
#     yield (x, y)


In [6]:
def data_type():
    return tf.float16 

init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 1
num_layers = 1
num_steps = 2
hidden_size = 2
max_epoch = 1
max_max_epoch = 1
keep_prob = 1.0
lr_decay = 0.5
batch_size = 20
vocab_size = 10000

In [30]:
# loads / create the data to be learned
data = 'Felipe Farias|'*100
data = data.lower()
# remove accents
# data = normalize('NFKD', data).encode('ASCII','ignore')
data = np.array(list(data))

In [31]:
data

array(['f', 'e', 'l', ..., 'a', 's', '|'], 
      dtype='<U1')

In [32]:
# Converts string to symbols
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has {:,} characters, {} unique.'.format(data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
for k,v in char_to_ix.items():
    print('{} ({}) -> {}'.format(k, ord(k),v))

data has 1,400 characters, 10 unique.
a (97) -> 0
i (105) -> 1
e (101) -> 5
r (114) -> 6
s (115) -> 3
| (124) -> 7
  (32) -> 4
l (108) -> 9
p (112) -> 8
f (102) -> 2


In [33]:
class PTBModel(object):
	"""The PTB model."""

	def __init__(self, is_training, config):
		self.batch_size = batch_size = config.batch_size
		self.num_steps = num_steps = config.num_steps
		size = config.hidden_size
		vocab_size = config.vocab_size

		self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
		self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])

		# Slightly better results can be obtained with forget gate biases
		# initialized to 1 but the hyperparameters of the model would need to be
		# different than reported in the paper.
		lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0, state_is_tuple=True)
		if is_training and config.keep_prob < 1:
			lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
					lstm_cell, output_keep_prob=config.keep_prob)
		cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers, state_is_tuple=True)

		self._initial_state = cell.zero_state(batch_size, data_type())

		with tf.device("/cpu:0"):
			embedding = tf.get_variable(
					"embedding", [vocab_size, size], dtype=data_type())
			inputs = tf.nn.embedding_lookup(embedding, self._input_data)

		if is_training and config.keep_prob < 1:
			inputs = tf.nn.dropout(inputs, config.keep_prob)

		# Simplified version of tensorflow.models.rnn.rnn.py's rnn().
		# This builds an unrolled LSTM for tutorial purposes only.
		# In general, use the rnn() or state_saving_rnn() from rnn.py.
		#
		# The alternative version of the code below is:
		#
		# from tensorflow.models.rnn import rnn
		# inputs = [tf.squeeze(input_, [1])
		#           for input_ in tf.split(1, num_steps, inputs)]
		# outputs, state = rnn.rnn(cell, inputs, initial_state=self._initial_state)
		outputs = []
		state = self._initial_state
		with tf.variable_scope("RNN"):
			for time_step in range(num_steps):
				if time_step > 0: tf.get_variable_scope().reuse_variables()
				(cell_output, state) = cell(inputs[:, time_step, :], state)
				outputs.append(cell_output)

		output = tf.reshape(tf.concat(1, outputs), [-1, size])
		softmax_w = tf.get_variable(
				"softmax_w", [size, vocab_size], dtype=data_type())
		softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=data_type())
		logits = tf.matmul(output, softmax_w) + softmax_b
		loss = tf.nn.seq2seq.sequence_loss_by_example(
				[logits],
				[tf.reshape(self._targets, [-1])],
				[tf.ones([batch_size * num_steps], dtype=data_type())])
		self._cost = cost = tf.reduce_sum(loss) / batch_size
		self._final_state = state

		if not is_training:
			return

		self._lr = tf.Variable(0.0, trainable=False)
		tvars = tf.trainable_variables()
		grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
																			config.max_grad_norm)
		optimizer = tf.train.GradientDescentOptimizer(self._lr)
		self._train_op = optimizer.apply_gradients(zip(grads, tvars))

		self._new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
		self._lr_update = tf.assign(self._lr, self._new_lr)

	def assign_lr(self, session, lr_value):
		session.run(self._lr_update, feed_dict={self._new_lr: lr_value})

	@property
	def input_data(self):
		return self._input_data

	@property
	def targets(self):
		return self._targets

	@property
	def initial_state(self):
		return self._initial_state

	@property
	def cost(self):
        return self._cost

	@property
	def final_state(self):
		return self._final_state

	@property
	def lr(self):
		return self._lr

	@property
	def train_op(self):
		return self._train_op


TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-33-9c847d60009c>, line 93)

In [10]:
def run_epoch(session, model, data, eval_op, verbose=False):
    """Runs the model on the given data."""
    epoch_size = ((len(data) // model.batch_size) - 1) // model.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = session.run(model.initial_state)
    for step, (x, y) in enumerate(reader.ptb_iterator(data, model.batch_size,
                                                    model.num_steps)):
    fetches = [model.cost, model.final_state, eval_op]
    feed_dict = {}
    feed_dict[model.input_data] = x
    feed_dict[model.targets] = y
    for i, (c, h) in enumerate(model.initial_state):
        feed_dict[c] = state[i].c
        feed_dict[h] = state[i].h
    cost, state, _ = session.run(fetches, feed_dict)
    costs += cost
    iters += model.num_steps

    if verbose and step % (epoch_size // 10) == 10:
        print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / epoch_size, np.exp(costs / iters),
             iters * model.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)

In [ ]:
def main(_):
    if not FLAGS.data_path:
    raise ValueError("Must set --data_path to PTB data directory")

    raw_data = reader.ptb_raw_data(FLAGS.data_path)
    train_data, valid_data, test_data, _ = raw_data

    config = get_config()
    eval_config = get_config()
    eval_config.batch_size = 1
    eval_config.num_steps = 1

    with tf.Graph().as_default(), tf.Session() as session:
        initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True, config=config)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False, config=config)
        mtest = PTBModel(is_training=False, config=eval_config)

    tf.initialize_all_variables().run()

    for i in range(config.max_max_epoch):
        lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
        m.assign_lr(session, config.learning_rate * lr_decay)

        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
        valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    print("Test Perplexity: %.3f" % test_perplexity)

In [ ]:
tf.app.run()
